In [3]:
from elasticsearch import Elasticsearch

In [4]:
# TODO: Add authentication for elasticsearch?

es_host = "http://localhost:9200"
# es_user = "elastic"
# es_password = "1234"

es = Elasticsearch(
    hosts=es_host # "http://localhost:9200"
    # http_auth=(es_user, es_password)   # credentials for basic authentication
)

es.info()   # check if connection was successful

/home/phansen/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '6c142f83f762',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'EpHkTVwuS9-4FxpMFLDV7g',
 'version': {'number': '7.17.7',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '78dcaaa8cee33438b91eca7f5c7f56a70fec9e80',
  'build_date': '2022-10-17T15:29:54.167373105Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### Creating a new index

In [5]:
index_name = "lyrics_mood_classification"

es.indices.create(index=index_name,
                  mappings={
                      "dynamic": "strict",
                      "properties": {
                          "song_name":    {"type": "text"},
                          "artist_name":  {"type": "text"},
                          "lyrics":   {"type": "text"},
                          "mood":   {"type": "text"}
                      }
                  })

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [lyrics_mood_classification/TgCE1tWIRcKGzAmMrskg4Q] already exists')

In [6]:
es.indices.get(index=index_name)

{'lyrics_mood_classification': {'aliases': {},
  'mappings': {'dynamic': 'strict',
   'properties': {'artist_name': {'type': 'text'},
    'lyrics': {'type': 'text'},
    'mood': {'type': 'text'},
    'song_name': {'type': 'text'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'lyrics_mood_classification',
    'creation_date': '1671613588420',
    'number_of_replicas': '1',
    'uuid': 'TgCE1tWIRcKGzAmMrskg4Q',
    'version': {'created': '7170799'}}}}}

### Create a new document

In [8]:
import scrape_genius_lyrics as sgl

artist_name = "Eminem"
song_name = "Mockingbird"
lyrics = sgl.scrape_lyrics(artist_name, song_name)
mood = "sad"

Searching for "Eminem" by Mockingbird...
Done.
Song found.


In [20]:
# Initialize id counter
id_counter = 0

In [28]:
# Create example document
es.create(index=index_name,
          id=str(id_counter),
          document={
            "song_name": song_name,
            "artist_name":  artist_name,
            "lyrics": lyrics,
            "mood": mood
          })
id_counter += 1

In [23]:
es.search(index=index_name)

{'took': 356,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'lyrics_mood_classification',
    '_type': '_doc',
    '_id': '0',
    '_score': 1.0,
    '_source': {'song_name': 'Mockingbird',
     'artist_name': 'Eminem',
     'lyrics': "TranslationsTürkçePortuguêsItalianoDeutschFrançaisEnglishMockingbird Lyrics[Intro]\nYeah\nI know sometimes\nThings may not always make sense to you right now\nBut ayy\nWhat'd Daddy always tell you?\nStraighten up, little soldier, stiffen up that upper lip\nWhat you cryin' about?\nYou got me\n\n[Verse 1]\nHailie, I know you miss your Mom, and I know you miss your Dad\nWhen I'm gone, but I'm tryna give you the life that I never had\nI can see you're sad, even when you smile, even when you laugh\nI can see it in your eyes, deep inside you wanna cry\n'Cause you're scared, I ain't there, Daddy's witchu in your prayers\nNo m

In [30]:
# Search song in es index
result = es.search(index=index_name, query={"bool": {"must": [
        {"match": {"song_name": song_name}}, {"match": {"artist_name": artist_name}}]}})
print(result)

/home/phansen/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4, 'relation': 'eq'}, 'max_score': 0.21072102, 'hits': [{'_index': 'lyrics_mood_classification', '_type': '_doc', '_id': '0', '_score': 0.21072102, '_source': {'song_name': 'Mockingbird', 'artist_name': 'Eminem', 'lyrics': "TranslationsTürkçePortuguêsItalianoDeutschFrançaisEnglishMockingbird Lyrics[Intro]\nYeah\nI know sometimes\nThings may not always make sense to you right now\nBut ayy\nWhat'd Daddy always tell you?\nStraighten up, little soldier, stiffen up that upper lip\nWhat you cryin' about?\nYou got me\n\n[Verse 1]\nHailie, I know you miss your Mom, and I know you miss your Dad\nWhen I'm gone, but I'm tryna give you the life that I never had\nI can see you're sad, even when you smile, even when you laugh\nI can see it in your eyes, deep inside you wanna cry\n'Cause you're scared, I ain't there, Daddy's witchu in your prayers\nNo more cryin', wipe them